

## 5. Implementation Code



In [2]:
import numpy as np

def propagate_vectorized(w, b, X):
    """
    Computes Forward Propagation for the entire dataset X without loops.
    
    Arguments:
    w -- weights, shape (n_x, 1)
    b -- bias, scalar
    X -- data, shape (n_x, m)
    
    Returns:
    A -- The vector of predictions, shape (1, m)
    """
    
    # 1. Linear Step (Z)
    # np.dot computes w.T * X
    # Python broadcasts 'b' to every element of the resulting vector
    Z = np.dot(w.T, X) + b
    
    # 2. Activation Step (A)
    # Applies sigmoid to every element of Z at once
    A = 1 / (1 + np.exp(-Z))
    
    return A

# --- VERIFICATION ---
if __name__ == "__main__":
    n_x, m = 3, 5
    w = np.ones((n_x, 1))
    b = 1.0
    X = np.ones((n_x, m))
    
    A = propagate_vectorized(w, b, X)
    
    print(f"Output Shape: {A.shape} (Should be 1, {m})")
    print(f"Values: {A}")

Output Shape: (1, 5) (Should be 1, 5)
Values: [[0.98201379 0.98201379 0.98201379 0.98201379 0.98201379]]


In [5]:
# Generic Logistic Regression Implementation (Vectorized)

import numpy as np

class LogisticRegressionModel:
    def __init__(self):
        self.w = None
        self.b = None
        self.costs = []

    def sigmoid(self, z):
        """
        Compute the sigmoid of z
        """
        return 1 / (1 + np.exp(-z))

    def initialize_parameters(self, n_x):
        """
        Argument:
        n_x -- size of the input layer (number of features)
        
        Returns:
        w -- initialized vector of shape (n_x, 1)
        b -- initialized scalar (corresponds to the bias)
        """
        self.w = np.zeros((n_x, 1))
        self.b = 0.0
        return self.w, self.b

    def propagate(self, w, b, X, Y):
        """
        Implement the cost function and its gradient for the propagation explained above

        Arguments:
        w -- weights, a numpy array of size (n_x, 1)
        b -- bias, a scalar
        X -- data of size (n_x, m)
        Y -- true "label" vector (containing 0 if non-cat, 1 if cat) of size (1, m)

        Return:
        cost -- negative log-likelihood cost for logistic regression
        dw -- gradient of the loss with respect to w, thus same shape as w
        db -- gradient of the loss with respect to b, thus same shape as b
        """
        m = X.shape[1]
        
        # --- FORWARD PROPAGATION (FROM X TO COST) ---
        # Matrix Op: Z = w.T * X + b
        Z = np.dot(w.T, X) + b
        A = self.sigmoid(Z)                                 
        
        # Cost Function (Log Loss)
        # We add 1e-15 to avoid log(0) errors
        cost = (-1/m) * np.sum(Y * np.log(A + 1e-15) + (1-Y) * np.log(1-A + 1e-15))
        cost = np.squeeze(cost)  # Ensure cost is a scalar (e.g. 17) not an array [[17]]
        
        # --- BACKWARD PROPAGATION (TO FIND GRADIENTS) ---
        dZ = A - Y
        dw = (1/m) * np.dot(X, dZ.T)
        db = (1/m) * np.sum(dZ)
        
        grads = {"dw": dw, "db": db}
        
        return grads, cost

    def optimize(self, w, b, X, Y, num_iterations, learning_rate, print_cost=False):
        """
        This function optimizes w and b by running a gradient descent algorithm
        """
        costs = []
        
        for i in range(num_iterations):
            # 1. Calculate Gradients and Cost
            grads, cost = self.propagate(w, b, X, Y)
            
            # 2. Retrieve derivatives
            dw = grads["dw"]
            db = grads["db"]
            
            # 3. Update Parameters (Gradient Descent Step)
            w = w - learning_rate * dw
            b = b - learning_rate * db
            
            # Record the cost
            if i % 100 == 0:
                costs.append(cost)
                if print_cost:
                    print(f"Cost after iteration {i}: {cost:.6f}")
        
        self.w = w
        self.b = b
        self.costs = costs
        
        return w, b, grads, costs

    def predict(self, X):
        """
        Predict whether the label is 0 or 1 using learned logistic regression parameters (w, b)
        """
        m = X.shape[1]
        Y_prediction = np.zeros((1, m))
        w = self.w.reshape(X.shape[0], 1)
        
        # Compute vector A
        A = self.sigmoid(np.dot(w.T, X) + self.b)
        
        # Convert probabilities A[0,i] to actual predictions p[0,i]
        for i in range(A.shape[1]):
            if A[0, i] > 0.5:
                Y_prediction[0, i] = 1
            else:
                Y_prediction[0, i] = 0
        
        return Y_prediction

    def fit(self, X_train, Y_train, num_iterations=2000, learning_rate=0.5, print_cost=True):
        """
        Main function to train the model
        """
        # Initialize
        n_x = X_train.shape[0]
        self.initialize_parameters(n_x)
        
        # Optimize
        self.optimize(self.w, self.b, X_train, Y_train, num_iterations, learning_rate, print_cost)
        
        return self

# --- USAGE EXAMPLE ---
if __name__ == "__main__":
    # 1. Synthetic Data Generation
    # 2 Features, 1000 Examples
    m_train = 1000
    n_features = 2
    
    # Random input data (2, 1000)
    X = np.random.randn(n_features, m_train)
    
    # Generate labels: If sum of features > 0, label is 1, else 0
    Y = (np.sum(X, axis=0, keepdims=True) > 0).astype(int)
    
    # 2. Create and Train Model
    print("--- Training Logistic Regression ---")
    model = LogisticRegressionModel()
    model.fit(X, Y, num_iterations=5000, learning_rate=0.01, print_cost=True)
    
    # 3. Predict on new data
    X_test = np.array([[1.5, -2.0], [1.5, -2.0]]) # 2 examples
    print("\n--- Test Prediction ---")
    predictions = model.predict(X_test)
    print(f"Test Input:\n{X_test}")
    print(f"Predictions: {predictions}")

--- Training Logistic Regression ---
Cost after iteration 0: 0.693147
Cost after iteration 100: 0.564679
Cost after iteration 200: 0.484211
Cost after iteration 300: 0.430080
Cost after iteration 400: 0.391193
Cost after iteration 500: 0.361773
Cost after iteration 600: 0.338612
Cost after iteration 700: 0.319809
Cost after iteration 800: 0.304166
Cost after iteration 900: 0.290897
Cost after iteration 1000: 0.279458
Cost after iteration 1100: 0.269466
Cost after iteration 1200: 0.260640
Cost after iteration 1300: 0.252768
Cost after iteration 1400: 0.245689
Cost after iteration 1500: 0.239277
Cost after iteration 1600: 0.233434
Cost after iteration 1700: 0.228078
Cost after iteration 1800: 0.223145
Cost after iteration 1900: 0.218581
Cost after iteration 2000: 0.214342
Cost after iteration 2100: 0.210390
Cost after iteration 2200: 0.206695
Cost after iteration 2300: 0.203227
Cost after iteration 2400: 0.199966
Cost after iteration 2500: 0.196890
Cost after iteration 2600: 0.193983
Cos